In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Milvus
from langchain.document_loaders import WebBaseLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
import os

loader = TextLoader('./datasets/www.lu.ch.txt')

docs = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
docs = text_splitter.split_documents(docs)


Created a chunk of size 1668, which is longer than the specified 1024


In [11]:
connection_args={
        "uri": "https://in03-5052868020ac71b.api.gcp-us-west1.zillizcloud.com",
        "user": "vaclav@pechtor.ch",
        "token": "",
        "secure": True
    }

os.environ["OPENAI_API_KEY"] = ''

In [12]:
embeddings = OpenAIEmbeddings()

vector_store = Milvus.from_documents(
    docs,
    embedding=embeddings,
    connection_args=connection_args
)


In [15]:
query = "Wann trifft sich der Kantonsrat nach den Wahlen?"
docs = vector_store.similarity_search(query)

print(docs)


[Document(page_content='Der Kantonsrat Luzern trifft sich nach den Wahlen vom April am Montag, 19. Juni 2023 erstmals in seiner neuen Zusammensetzung zur konstituierenden Sitzung, dabei wird auch das Präsidium neu gewählt. Danach beginnt die 1. Session der Legislatur 2023-2027. Der Kantonsrat tagt am 19./20. und 26. Juni 2023. Traktandiert ist unter anderem der Sonderkredit für die Erweiterung und Erneuerung der Kantonsschule Sursee für rund 69,5 Millionen Franken.\xa0\nZum Abschluss der Juni-Session werden die drei abtretenden Regierungsräte Guido Graf, Marcel Schwerzmann und Paul Winiker verabschiedet.\nAuf der\xa0Traktandenliste\xa0stehen über 130 Geschäfte.\nZum Live-Stream im Vollbild.\nDie Luzerner Stimmberechtigten haben mit 64 Prozent Ja-Stimmen den Sonderkredit in der Höhe von 70,6 Millionen Franken für die Ost- und Westumfahrung des historischen Ortskerns von Beromünster genehmigt. \nMedienmitteilung vom 18. Juni 2023', metadata={'source': './datasets/www.lu.ch.txt'}), Docume

In [16]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_reduce", return_intermediate_steps=True)
query = "Wann trifft sich der Kantonsrat nach den Wahlen?"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': [' Der Kantonsrat Luzern trifft sich nach den Wahlen vom April am Montag, 19. Juni 2023 erstmals in seiner neuen Zusammensetzung zur konstituierenden Sitzung.',
  ' None',
  ' LIVE: Juni-Session des Kantonsrates',
  ' None'],
 'output_text': ' Der Kantonsrat Luzern trifft sich nach den Wahlen vom April am Montag, 19. Juni 2023 erstmals in seiner neuen Zusammensetzung zur konstituierenden Sitzung.\nSOURCES: ./datasets/www.lu.ch.txt'}